In [ ]:
# Importing packages

import pandas as pd
import numpy  as np
import seaborn as sns
import datetime 
import chardet


In [ ]:
# detect files encoding
with open("OBD_2024_I.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # Read first 100,000 bytes
    print(result)

In [ ]:
# Importing data
oko_2024_df = pd.read_csv("OBD_2024_I.csv", delimiter=";", encoding = result["encoding"], low_memory=False)
oko_2023_df= pd.read_csv("OBD_2023_I.csv", delimiter=";", encoding = result["encoding"], low_memory=False)
oko_2020_df = pd.read_csv("OBD_2020_II.csv", delimiter=";", encoding = result["encoding"], low_memory=False)
tbau_df = pd.read_csv("tBaustoff_with_OBD_mapping.csv", delimiter=";", encoding = result["encoding"], low_memory=False)


In [ ]:
# pollutants_df = pd.read_csv("pollutant_combinations.csv", delimiter=";", encoding = result["encoding"], low_memory=False)